In [5]:
!gsutil cp "gs://fc-secure-089622a2-ae22-4dc8-ad01-634bd7634d5f/keys/aou_wgs_vumc_prod.json" .
!gsutil cp "gs://fc-secure-062111d2-d6d9-4613-85c0-1809d891faa6/notebooks/filter_set_samples_create_metrics.functions.sql" .

Copying gs://fc-secure-089622a2-ae22-4dc8-ad01-634bd7634d5f/keys/aou_wgs_vumc_prod.json...
/ [1 files][  2.3 KiB/  2.3 KiB]                                                
Operation completed over 1 objects/2.3 KiB.                                      
Copying gs://fc-secure-062111d2-d6d9-4613-85c0-1809d891faa6/notebooks/filter_set_samples_create_metrics.functions.sql...
/ [1 files][  1.1 KiB/  1.1 KiB]                                                
Operation completed over 1 objects/1.1 KiB.                                      


In [ ]:
! # Notes --- 
! # when run on the Charlie callset, the original version of this query timed out, 
! # so it was updated to be 3 different queries
! # The first two populate the table with duplicates by querying the prepare VET_DATA table twice (or however many times you need to get all the data)
! # The third query groups on sample id (and filter--not sure why since it should be the same) to remove duplicates and sum everything up
! # 
! # The best version of this would lkely be an intial query (or set of queries) that creates a temp table for the innter select statement

In [ ]:
import time

from google.cloud import bigquery
from google.cloud.bigquery.job import QueryJobConfig
from google.oauth2 import service_account
from pathlib import Path

FQ_PREFIX = "aou-genomics-curation-prod.aou_wgs.charlie"
NAME_OF_FILTER_SET = "charlie"

sa_key_path = "aou_wgs_vumc_prod.json"

credentials = service_account.Credentials.from_service_account_file(
    sa_key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
job_config = QueryJobConfig(priority="INTERACTIVE",
                            use_query_cache=True)

query_project = "aou-genomics-curation-prod"
client = bigquery.Client(credentials=credentials,
                         project=query_project,
                         default_query_job_config=job_config)

f = open("filter_set_samples_create_metrics.functions.sql", "r")
sql = "\n".join(f.readlines())

sql += f"""CREATE OR REPLACE TABLE `{FQ_PREFIX}_sample_metrics` AS
SELECT "{NAME_OF_FILTER_SET}" filter_set_name,
       sample_id,
       count(1) variant_entries,
       SUM(CASE WHEN type = "del" THEN 1 ELSE 0 END) del_count,
       SUM(CASE WHEN type = "ins" THEN 1 ELSE 0 END) ins_count,
       SUM(CASE WHEN type = "snp" THEN 1 ELSE 0 END) snp_count,
       SUM(CASE WHEN type = "snp" AND titv = "ti" THEN 1 ELSE 0 END) ti_count, # TODO: minimize alleles
       SUM(CASE WHEN type = "snp" AND titv = "tv" THEN 1 ELSE 0 END) tv_count, # TODO: minimize alleles
       SUM(CASE WHEN type = "snp" AND gt_type = "het" THEN 1 ELSE 0 END) snp_het_count,
       SUM(CASE WHEN type = "snp" AND gt_type = "homvar" THEN 1 ELSE 0 END) snp_homvar_count,
       SUM(CASE WHEN type IN ("ins","del") AND gt_type = "het" THEN 1 ELSE 0 END) indel_het_count,
       SUM(CASE WHEN type IN ("ins","del") AND gt_type = "homvar" THEN 1 ELSE 0 END) indel_homvar_count,
       COUNTIF(not in_gnomad) singleton,
       null AS pass_qc
    FROM (
        SELECT sample_id,
               type(ref, alt, call_GT) as type,
               CASE WHEN INSTR(call_GT, "0") > 0 THEN "het" ELSE "homvar" END as gt_type,
               titv(ref, alt) as titv,
               CASE WHEN gnomad.location IS NULL THEN false ELSE true END in_gnomad
        FROM `{FQ_PREFIX}__VET_DATA` v
        LEFT JOIN `spec-ops-aou.gvs_public_reference_data.gnomad_v3_sites` gnomad ON (v.location = gnomad.location)
        WHERE call_GT != "./."
        AND v.location < 10000000000000) GROUP BY 1,2"""

start = time.time()
print("STARTING")
query = client.query(sql, job_config=job_config)
result = query.result()
print(f"COMPLETED {time.time() - start}s")


In [ ]:
import time

from google.cloud import bigquery
from google.cloud.bigquery.job import QueryJobConfig
from google.oauth2 import service_account
from pathlib import Path

FQ_PREFIX = "aou-genomics-curation-prod.aou_wgs.charlie"
NAME_OF_FILTER_SET = "charlie"

sa_key_path = "aou_wgs_vumc_prod.json"

credentials = service_account.Credentials.from_service_account_file(
    sa_key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
job_config = QueryJobConfig(priority="INTERACTIVE",
                            use_query_cache=True)

query_project = "aou-genomics-curation-prod"
client = bigquery.Client(credentials=credentials,
                         project=query_project,
                         default_query_job_config=job_config)

f = open("filter_set_samples_create_metrics.functions.sql", "r")
sql = "\n".join(f.readlines())

sql += f"""

INSERT `{FQ_PREFIX}_sample_metrics` (
    filter_set_name,
    sample_id,
    variant_entries,
    del_count, 
    ins_count,
    snp_count,
    ti_count,
    tv_count,
    snp_het_count,
    snp_homvar_count,
    indel_het_count,
    indel_homvar_count,
    singleton,
    pass_qc
    )
SELECT "{NAME_OF_FILTER_SET}" filter_set_name,
       sample_id,
       count(1) variant_entries,
       SUM(CASE WHEN type = "del" THEN 1 ELSE 0 END) del_count,
       SUM(CASE WHEN type = "ins" THEN 1 ELSE 0 END) ins_count,
       SUM(CASE WHEN type = "snp" THEN 1 ELSE 0 END) snp_count,
       SUM(CASE WHEN type = "snp" AND titv = "ti" THEN 1 ELSE 0 END) ti_count, # TODO: minimize alleles
       SUM(CASE WHEN type = "snp" AND titv = "tv" THEN 1 ELSE 0 END) tv_count, # TODO: minimize alleles
       SUM(CASE WHEN type = "snp" AND gt_type = "het" THEN 1 ELSE 0 END) snp_het_count,
       SUM(CASE WHEN type = "snp" AND gt_type = "homvar" THEN 1 ELSE 0 END) snp_homvar_count,
       SUM(CASE WHEN type IN ("ins","del") AND gt_type = "het" THEN 1 ELSE 0 END) indel_het_count,
       SUM(CASE WHEN type IN ("ins","del") AND gt_type = "homvar" THEN 1 ELSE 0 END) indel_homvar_count,
       COUNTIF(not in_gnomad) singleton,
       null AS pass_qc
    FROM (
        SELECT sample_id,
               type(ref, alt, call_GT) as type,
               CASE WHEN INSTR(call_GT, "0") > 0 THEN "het" ELSE "homvar" END as gt_type,
               titv(ref, alt) as titv,
               CASE WHEN gnomad.location IS NULL THEN false ELSE true END in_gnomad
        FROM `{FQ_PREFIX}__VET_DATA` v
        LEFT JOIN `spec-ops-aou.gvs_public_reference_data.gnomad_v3_sites` gnomad ON (v.location = gnomad.location)
        WHERE call_GT != "./."
        AND v.location >= 10000000000000
        AND v.location < 23000000000000) GROUP BY 1,2"""

start = time.time()
print("STARTING")
query = client.query(sql, job_config=job_config)
result = query.result()
print(f"COMPLETED {time.time() - start}s")


In [ ]:
import time

from google.cloud import bigquery
from google.cloud.bigquery.job import QueryJobConfig
from google.oauth2 import service_account
from pathlib import Path

FQ_PREFIX = "aou-genomics-curation-prod.aou_wgs.charlie"
NAME_OF_FILTER_SET = "charlie"

sa_key_path = "aou_wgs_vumc_prod.json"

credentials = service_account.Credentials.from_service_account_file(
    sa_key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
job_config = QueryJobConfig(priority="INTERACTIVE",
                            use_query_cache=True)

query_project = "aou-genomics-curation-prod"
client = bigquery.Client(credentials=credentials,
                         project=query_project,
                         default_query_job_config=job_config)

f = open("filter_set_samples_create_metrics.functions.sql", "r")
sql = "\n".join(f.readlines())

sql += f"""CREATE OR REPLACE TABLE `{FQ_PREFIX}_sample_metrics_agg` AS
SELECT "{NAME_OF_FILTER_SET}" filter_set_name,
       sample_id,
       SUM(variant_entries) variant_entries,
       SUM(CASE WHEN type = "del" THEN 1 ELSE 0 END) del_count,
       SUM(CASE WHEN type = "ins" THEN 1 ELSE 0 END) ins_count,
       SUM(CASE WHEN type = "snp" THEN 1 ELSE 0 END) snp_count,
       SUM(CASE WHEN type = "snp" AND titv = "ti" THEN 1 ELSE 0 END) ti_count, # TODO: minimize alleles
       SUM(CASE WHEN type = "snp" AND titv = "tv" THEN 1 ELSE 0 END) tv_count, # TODO: minimize alleles
       SUM(CASE WHEN type = "snp" AND gt_type = "het" THEN 1 ELSE 0 END) snp_het_count,
       SUM(CASE WHEN type = "snp" AND gt_type = "homvar" THEN 1 ELSE 0 END) snp_homvar_count,
       SUM(CASE WHEN type IN ("ins","del") AND gt_type = "het" THEN 1 ELSE 0 END) indel_het_count,
       SUM(CASE WHEN type IN ("ins","del") AND gt_type = "homvar" THEN 1 ELSE 0 END) indel_homvar_count,
       SUM(singleton) singleton,
       null AS pass_qc
    FROM `{FQ_PREFIX}_sample_metrics` GROUP BY 1,2"""

start = time.time()
print("STARTING")
query = client.query(sql, job_config=job_config)
result = query.result()
print(f"COMPLETED {time.time() - start}s")


In [2]:
import time

from google.cloud import bigquery
from google.cloud.bigquery.job import QueryJobConfig
from google.oauth2 import service_account
from pathlib import Path

FQ_PREFIX = "aou-genomics-curation-prod.aou_wgs.charlie"
NAME_OF_FILTER_SET = "charlie"
DESTINATION = f"{FQ_PREFIX}_samples_stats"

sa_key_path = "aou_wgs_vumc_prod.json"

credentials = service_account.Credentials.from_service_account_file(
    sa_key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
job_config = QueryJobConfig(priority="INTERACTIVE",
                            use_query_cache=True,
                            destination=f"{DESTINATION}")

query_project = "aou-genomics-curation-prod"
client = bigquery.Client(credentials=credentials,
                         project=query_project,
                         default_query_job_config=job_config)


sql = f"""WITH fss AS (
  SELECT *, 
         (ins_count / del_count) as ins_del_ratio, 
         (ti_count / tv_count) as ti_tv_ratio, 
         (snp_het_count / snp_homvar_count) snp_het_homvar_ratio, 
         (indel_het_count / indel_homvar_count) as indel_het_homvar_ratio
  FROM `{FQ_PREFIX}_sample_metrics_agg`
  WHERE filter_set_name = '{NAME_OF_FILTER_SET}'),
medians AS ( 
    SELECT 
        `bqutil`.fn.median(ARRAY_AGG(del_count IGNORE NULLS)) as m_del_count,
        `bqutil`.fn.median(ARRAY_AGG(ins_count IGNORE NULLS)) as m_ins_count,
        `bqutil`.fn.median(ARRAY_AGG(snp_count IGNORE NULLS)) as m_snp_count,
        `bqutil`.fn.median(ARRAY_AGG(singleton IGNORE NULLS)) as m_singleton,
        `bqutil`.fn.median(ARRAY_AGG(ins_del_ratio IGNORE NULLS)) as m_ins_del_ratio,
        `bqutil`.fn.median(ARRAY_AGG(ti_tv_ratio IGNORE NULLS)) as m_ti_tv_ratio,
        `bqutil`.fn.median(ARRAY_AGG(snp_het_homvar_ratio IGNORE NULLS)) as m_snp_het_homvar_ratio,
        `bqutil`.fn.median(ARRAY_AGG(indel_het_homvar_ratio IGNORE NULLS)) as m_indel_het_homvar_ratio        
    FROM fss),
mads AS (
    SELECT 
        `bqutil`.fn.median(ARRAY_AGG(ABS(del_count - m_del_count) IGNORE NULLS)) as mad_del_count,
        `bqutil`.fn.median(ARRAY_AGG(ABS(ins_count - m_ins_count) IGNORE NULLS)) as mad_ins_count,
        `bqutil`.fn.median(ARRAY_AGG(ABS(snp_count - m_snp_count) IGNORE NULLS)) as mad_snp_count,
        `bqutil`.fn.median(ARRAY_AGG(ABS(singleton - m_singleton) IGNORE NULLS)) as mad_singleton,
        `bqutil`.fn.median(ARRAY_AGG(ABS(ins_del_ratio - m_ins_del_ratio) IGNORE NULLS)) as mad_ins_del_ratio,
        `bqutil`.fn.median(ARRAY_AGG(ABS(ti_tv_ratio - m_ti_tv_ratio) IGNORE NULLS)) as mad_ti_tv_ratio,
        `bqutil`.fn.median(ARRAY_AGG(ABS(snp_het_homvar_ratio - m_snp_het_homvar_ratio) IGNORE NULLS)) as mad_snp_het_homvar_ratio,
        `bqutil`.fn.median(ARRAY_AGG(ABS(indel_het_homvar_ratio - m_indel_het_homvar_ratio) IGNORE NULLS)) as mad_indel_het_homvar_ratio
    FROM fss
    CROSS JOIN medians 
    WHERE filter_set_name = '{NAME_OF_FILTER_SET}')

SELECT 
    fss.sample_id,
    si.sample_name,
    del_count, m_del_count, mad_del_count,
    CASE WHEN del_count BETWEEN m_del_count - 4*mad_del_count AND m_del_count + 4*mad_del_count THEN true ELSE false END pass_del_count,
    
    ins_count, m_ins_count, mad_ins_count,
    CASE WHEN ins_count BETWEEN m_ins_count - 4*mad_ins_count AND m_ins_count + 4*mad_ins_count THEN true ELSE false END pass_ins_count,

    snp_count, m_snp_count, mad_snp_count,
    CASE WHEN snp_count BETWEEN m_snp_count - 4*mad_snp_count AND m_snp_count + 4*mad_snp_count THEN true ELSE false END pass_snp_count,

    singleton, m_singleton, mad_singleton,
    CASE WHEN singleton BETWEEN m_singleton - 8*mad_singleton AND m_singleton + 8*mad_singleton THEN true ELSE false END pass_singleton,
    
    ins_del_ratio, m_ins_del_ratio, mad_ins_del_ratio,
    CASE WHEN ins_del_ratio BETWEEN m_ins_del_ratio - 4*mad_ins_del_ratio AND m_ins_del_ratio + 4*mad_ins_del_ratio THEN true ELSE false END pass_ins_del_ratio,

    ti_tv_ratio, m_ti_tv_ratio, mad_ti_tv_ratio,
    CASE WHEN ti_tv_ratio BETWEEN m_ti_tv_ratio - 4*mad_ti_tv_ratio AND m_ti_tv_ratio + 4*mad_ti_tv_ratio THEN true ELSE false END pass_ti_tv_ratio,

    snp_het_homvar_ratio, m_snp_het_homvar_ratio, mad_snp_het_homvar_ratio,
    CASE WHEN snp_het_homvar_ratio BETWEEN m_snp_het_homvar_ratio - 4*mad_snp_het_homvar_ratio AND m_snp_het_homvar_ratio + 4*mad_snp_het_homvar_ratio THEN true ELSE false END pass_snp_het_homvar_ratio,

    indel_het_homvar_ratio, m_indel_het_homvar_ratio, mad_indel_het_homvar_ratio,
    CASE WHEN indel_het_homvar_ratio BETWEEN m_indel_het_homvar_ratio - 4*mad_indel_het_homvar_ratio AND m_indel_het_homvar_ratio + 4*mad_indel_het_homvar_ratio THEN true ELSE false END pass_indel_het_homvar_ratio,    
FROM fss
JOIN `{FQ_PREFIX}__SAMPLES` si ON (fss.sample_id = si.sample_id)
CROSS JOIN medians 
CROSS JOIN mads
order by 1"""

start = time.time()
print("STARTING")
query = client.query(sql, job_config=job_config)
result = query.result()
print(f"COMPLETED {time.time() - start}s")


STARTING
COMPLETED 9.259913206100464s


In [1]:
!gsutil cp "gs://fc-secure-089622a2-ae22-4dc8-ad01-634bd7634d5f/keys/aou_wgs_vumc_prod.json" local.service_account.json
! export GOOGLE_APPLICATION_CREDENTIALS=local.service_account.json
!gcloud auth activate-service-account --key-file=local.service_account.json


Copying gs://fc-secure-089622a2-ae22-4dc8-ad01-634bd7634d5f/keys/aou_wgs_vumc_prod.json...
/ [1 files][  2.3 KiB/  2.3 KiB]                                                
Operation completed over 1 objects/2.3 KiB.                                      
Activated service account credentials for: [awardee-drcbroad@all-of-us-rdr-prod.iam.gserviceaccount.com]


In [12]:
from google.cloud import bigquery
from google.oauth2 import service_account

sa_key_path = "aou_wgs_vumc_prod.json"
project = "aou-genomics-curation-prod"
bucket_name = "prod-drc-broad"
dataset_id = "aou_wgs"
table_id = "charlie_samples_stats"

credentials = service_account.Credentials.from_service_account_file(
    sa_key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
job_config = QueryJobConfig(priority="INTERACTIVE",
                            use_query_cache=True)

client = bigquery.Client(credentials=credentials,
                         project=project,
                         default_query_job_config=job_config)


destination_uri = f"gs://{bucket_name}/aou-wgs-charlie/aou-wgs-charlie_stats.csv"
dataset_ref = bigquery.DatasetReference(project, dataset_id)
table_ref = dataset_ref.table(table_id)

extract_job = client.extract_table(
    table_ref,
    destination_uri,
    location="US",
)  # API request
extract_job.result()  # Waits for job to complete.

print(
    "Exported {}:{}.{} to {}".format(project, dataset_id, table_id, destination_uri)
)


Exported aou-genomics-curation-prod:aou_wgs.charlie_samples_stats to gs://prod-drc-broad/aou-wgs-charlie/aou-wgs-charlie_stats.csv


In [14]:
! gsutil ls gs://prod-drc-broad/aou-wgs-charlie/aou-wgs-charlie_stats.csv

gs://prod-drc-broad/aou-wgs-charlie/aou-wgs-charlie_stats.csv


In [3]:
! gsutil ls gs://prod-drc-broad/aou-wgs-charlie/*-charlie.vcf.gz | wc -l

20039
